In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime
import glob


Bad key "text.kerning_factor" on line 4 in
C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
def check_file_existance(file_ids , tmpl):
    files = []
    path = tmpl.format(subject = '*')
    directory_files = glob.glob(path , recursive=True)

    for f_name in file_ids:
        filename = tmpl.format(subject = f_name)
        if filename in directory_files:
            files.append((f_name , filename))
        else:
            files.append((f_name , 'None'))
    return files


In [3]:
def loadData(files , from_date , to_date):
    total  =  []
    
    for i in range(len(files)):
        file_id = files[i][0]
        filename = files[i][1]


        df = pd.read_csv(filename)
        df = df.loc[ (df['Date'] >= from_date) & (df['Date'] < to_date)]
        df.insert(0,'Symbol',file_id)
        total.append(df)
    
    return total

In [4]:
def create_change_df_baseDate(total):
    for df in total:
        df.dropna(inplace = True) 
        df['Close1']  = df.Close.shift()
        df['Change'] = (df.Close - df.Close1) * 100 /  df.Close         
        df.dropna(inplace = True)
    dataset = pd.concat(total)
    dataset = dataset.pivot(index='Date', columns='Symbol', values='Change')
    
    
    # column name with Null values
    # because some symbols have price history less than the others 
    imperfect_symbols = dataset.columns[dataset.isna().any()].tolist()
    
    # we remove the symbols that haven't compelete range of price
    for symbol in imperfect_symbols:
        if symbol in dataset.columns:
            dataset.drop(symbol , axis = 1, inplace = True)
        
        
    return dataset


def create_change_df_baseSymbol(total):
    for df in total:
        df.dropna(inplace = True) 
        df['Close1']  = df.Close.shift()
        df['Change'] = (df.Close - df.Close1) * 100 /  df.Close         
        df.dropna(inplace = True)
    dataset = pd.concat(total)
    dataset = dataset.pivot(index='Symbol', columns='Date', values='Change')
    
    
    #symbol name with Null values
    # because some symbols have price history less than the others 
    imperfect_symbols = dataset[dataset.isna().any(axis=1)].index.tolist()
    
    # we remove the symbols that haven't compelete range of price
    for symbol in imperfect_symbols:
        if symbol in dataset.index:
            dataset.drop(symbol , axis = 0, inplace = True)
        
        
    return dataset

In [5]:
df_info  = pd.read_csv('SPindex.csv')
df_info = df_info.set_index('Symbol')
df_info.head()

,Security,GICS Sector,GICS Sub Industry,CIK
Symbol,,,,
MMM,3M Company,Industrials,Industrial Conglomerates,66740
ABT,Abbott Laboratories,Health Care,Health Care Equipment,1800
ABBV,AbbVie Inc.,Health Care,Pharmaceuticals,1551152
ABMD,ABIOMED Inc,Health Care,Health Care Equipment,815094
ACN,Accenture plc,Information Technology,IT Consulting & Other Services,1467373


In [6]:
symbol_names = list(df_info.index)
files = check_file_existance(symbol_names , 'stocks\{subject}.csv')
total = loadData(files , '2015-01-01' , '2020-04-01')

In [7]:
data_perSymbol = create_change_df_baseSymbol(total)
data_perSymbol.to_csv('stocks/total_data_changePercent_Symbol.csv')
data_perSymbol.head()

Date,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-12,2015-01-13,2015-01-14,2015-01-15,2015-01-16,...,2020-03-18,2020-03-19,2020-03-20,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-30,2020-03-31
Symbol,,,,,,,,,,,,,,,,,,,,,
A,-1.909553,-1.582437,1.309825,2.910244,-0.739096,-1.196708,-1.415933,-1.254475,-2.762439,0.627455,...,-2.279657,-4.729320,1.053261,-4.876125,5.417912,2.898551,6.402606,-3.962766,2.421900,-1.466064
AAL,-0.055677,-1.583711,-0.056598,1.211333,-3.152632,-4.921336,1.626983,-2.003646,0.000000,0.803055,...,-33.733909,-13.216712,0.867053,-1.268294,26.364943,9.551658,1.724135,-11.538461,-14.612245,-0.492210
AAP,-1.335717,-0.070351,2.103682,0.868914,-0.511536,-0.862021,-3.362382,-1.942579,-1.898390,1.319997,...,-12.970478,-3.222050,-14.580838,4.420384,9.122479,4.415188,6.950165,-1.589963,0.932644,-3.407630
AAPL,-2.898825,0.009413,1.382829,3.700062,0.107136,-2.526318,0.880059,-0.382512,-2.789743,-0.783094,...,-2.509427,-0.772122,-6.778919,-2.170527,9.117794,-0.553927,4.999225,-4.319043,2.774613,-0.204493
ABBV,-1.918017,-0.497435,3.884653,1.035039,-2.812403,-0.030409,-3.738764,0.486658,-1.287965,2.556556,...,-2.801630,0.042216,-3.254865,-6.697674,4.430283,0.618474,8.018418,-1.596262,3.415736,1.246889


In [8]:
data_perdate = create_change_df_baseDate(total)
data_perdate.to_csv('stocks/total_data_changePercent_Date.csv')
data_perdate.head()

Symbol,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-06,-1.582437,-1.583711,-0.070351,0.009413,-0.497435,0.543355,-2.601712,-1.148645,-0.726557,-2.055869,...,-1.469888,-0.534465,-1.256626,-1.356453,-0.591547,-1.242681,-0.854920,-0.725683,-3.970977,-0.985217
2015-01-07,1.309825,-0.056598,2.103682,1.382829,3.884653,1.956950,3.084758,0.804282,2.055800,0.815640,...,0.047394,1.003090,2.526315,1.264887,0.782557,3.207523,2.426691,2.483274,0.945537,2.022517
2015-01-08,2.910244,1.211333,0.868914,3.700062,1.035039,0.227789,4.312116,2.014015,1.502001,2.482169,...,2.178949,1.637214,2.409415,2.679220,0.748961,1.712188,1.050527,2.091202,1.380078,1.516528
2015-01-09,-0.739096,-3.152632,-0.511536,0.107136,-2.812403,0.870965,-1.431917,-1.061946,-0.111382,-1.503343,...,-0.092810,-0.141156,-1.478398,-0.582667,-1.922534,-0.176964,-1.463500,1.440283,-2.877974,0.158191
2015-01-12,-1.196708,-4.921336,-0.862021,-2.526318,-0.030409,-2.276476,0.389008,0.833701,-0.301639,-1.012364,...,-1.483394,-1.959478,-0.956755,-1.030173,-2.225435,-0.754359,0.236306,0.148776,-1.558845,-1.911566
